In [1]:
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from  keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import copy

import sklearn.model_selection
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

/home/ubuntu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("./train_cleaned_unfiltered.csv")
test = pd.read_csv("./test_cleaned_unfiltered.csv")
list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [5]:
list_sentences_test[0]

'yo bitch ja rule is more succesful then you will ever be whats up with you and hating you sad mofuckas i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother too fuckin white boys get things right next time'

In [17]:
sid.polarity_scores(list_sentences_test[0]).values()

[0.243, 0.631, 0.126, -0.9228]

In [22]:
test_vader = np.zeros((len(list_sentences_test), 4))

for i in tqdm(range(len(list_sentences_test))):
    test_vader[i] = sid.polarity_scores(list_sentences_test[i]).values()




100%|██████████| 153164/153164 [01:54<00:00, 1332.57it/s]


In [23]:
train_vader = np.zeros((len(list_sentences_train), 4))

for i in tqdm(range(len(list_sentences_train))):
    train_vader[i] = sid.polarity_scores(list_sentences_train[i]).values()

100%|██████████| 159571/159571 [02:06<00:00, 1256.79it/s]


In [24]:
train_vader[0:10]


array([[ 0.    ,  0.901 ,  0.099 ,  0.5574],
       [ 0.118 ,  0.71  ,  0.172 ,  0.2263],
       [ 0.083 ,  0.849 ,  0.068 , -0.1779],
       [ 0.059 ,  0.859 ,  0.082 ,  0.4019],
       [ 0.    ,  0.663 ,  0.337 ,  0.6808],
       [ 0.    ,  0.464 ,  0.536 ,  0.7964],
       [ 0.531 ,  0.469 ,  0.    , -0.7783],
       [ 0.123 ,  0.782 ,  0.095 , -0.1779],
       [ 0.128 ,  0.844 ,  0.028 , -0.7684],
       [ 0.    ,  1.    ,  0.    ,  0.    ]])

In [25]:
y[0:10]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [27]:
pd.DataFrame(np.column_stack([train["id"].values, train_vader])).to_csv("/home/ubuntu/kaggle/toxicity/team_ensembl/Florian/out/train_1803_FG_vader.csv", header= False, index= False)
pd.DataFrame(np.column_stack([test["id"].values, test_vader])).to_csv("/home/ubuntu/kaggle/toxicity/team_ensembl/Florian/out/test_1803_FG_vader.csv", header= False, index= False)   

In [28]:
X_train, X_val, y_train, y_val = train_test_split(train_vader, y, test_size=0.33)

In [29]:
class LR:
    
    def __init__(self):
        self.models = []

    def get_mdl(self, X, y):
        m = LogisticRegression(C=10, dual=True)
        return m.fit(X, y)
    
    def fit(self, X, y):
        if len(y.shape)>1:
            for i in range(y.shape[1]):
                m = self.get_mdl(X, y[:,i])
                self.models.append((m))
        else:
            m = self.get_mdl(X, y)
            self.models.append((m))
        
    def predict(self, X):
        preds = np.zeros((X.shape[0], len(self.models)))
        for i in range(len(self.models)):
            m = self.models[i]
            preds[:,i] = m.predict_proba(X)[:,1]
        return preds

In [30]:
l = LogisticRegression(C=10, dual=True)
lr = LR()
lr.fit(X_train, y_train)
pred = lr.predict(X_val)
print(roc_auc_score(y_val, pred))

0.8694636907508385
